# Software-Engineering
# Machine Learning Model

## Purpose
The purpose of the machine learning model is to predict the features according to the training sets that we get from the dublin bike api and google weather api.

 - to get the number of available bike of one specific station
 - to gat the number of available bike stands of one specific station

## Import the packages

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns


from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.options.mode.chained_assignment=None
%matplotlib inline
import os
import pymysql
pymysql.install_as_MySQLdb()
import pandas as pd
import sys
from datetime import datetime, date, timedelta


## Connect to the RDS database

In [3]:
def connect_to_database():
                                                                       
    conn = pymysql.connect(
        host="group20db2.cuvpbui26dwd.eu-west-1.rds.amazonaws.com",  # mysql服务器地址
        port=3306,  # 端口号
        user="group20",  # 用户名
        passwd="comp30830",  # 密码
        db="segroupproject",  # 数据库名称
        charset='utf8',  # 连接编码，根据需要填写
    )
    return conn

try:
    conn = connect_to_database()
    cur = conn.cursor()
    print('--- CONNECTED TO DATABASE SUCCESSFULLY ---')
    
    
# Exit if connection not working   
except Exception as e:
        sys.exit(e)

    

--- CONNECTED TO DATABASE SUCCESSFULLY ---


## Get data from the RDS database
### Get the bike information from the bike table




In [4]:
df = pd.read_sql_query("SELECT * FROM segroupproject.dynamic_bikeData", conn)

# Examine dataframe object, show first 10 rows
df.head(10)

,number,status,available_bike_stands,available_bikes,last_update,now_time
0,42,OPEN,27,3,2020-03-05 10:25:33,2020-03-05 10:28:01
1,30,OPEN,17,3,2020-03-05 10:20:14,2020-03-05 10:28:01
2,54,OPEN,2,31,2020-03-05 10:22:53,2020-03-05 10:28:01
3,108,OPEN,40,0,2020-03-05 10:23:32,2020-03-05 10:28:01
4,56,OPEN,0,40,2020-03-05 10:21:31,2020-03-05 10:28:01
5,6,OPEN,17,3,2020-03-05 10:18:19,2020-03-05 10:28:01
6,18,OPEN,26,4,2020-03-05 10:24:59,2020-03-05 10:28:01
7,32,OPEN,3,27,2020-03-05 10:24:38,2020-03-05 10:28:01
8,52,OPEN,27,5,2020-03-05 10:21:12,2020-03-05 10:28:01
9,48,OPEN,18,22,2020-03-05 10:25:12,2020-03-05 10:28:01


In [5]:
#show the dataframe has how many rows and columns
print("The dataset has %s rows and %s columns." % df.shape)

The dataset has 634871 rows and 6 columns.


### Get the weather information from the weather table

In [6]:
# Create dataframe and store data running SQL query
df_2 = pd.read_sql_query("SELECT * FROM segroupproject.weather", conn)

# Examine dataframe object, show first 50 rows
df_2.head(10)

,current,lon,lat,weather_id,main,description,icon,base,temp,feels_like,...,dt,sys_type,sys_id,sys_country,sunrise,sunset,timezone,id,name,cod
0,2020-03-05 10:52,-6.27,53.34,802,Clouds,scattered clouds,03d,stations,3.86,0.51,...,1583404828,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
1,2020-03-05 11:22,-6.27,53.34,803,Clouds,broken clouds,04d,stations,4.29,-0.39,...,1583407215,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
2,2020-03-05 11:52,-6.27,53.34,803,Clouds,broken clouds,04d,stations,4.47,-0.17,...,1583408505,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
3,2020-03-05 12:22,-6.27,53.34,803,Clouds,broken clouds,04d,stations,5.31,0.82,...,1583410536,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
4,2020-03-05 12:52,-6.27,53.34,803,Clouds,broken clouds,04d,stations,5.48,1.03,...,1583412184,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
5,2020-03-05 13:22,-6.27,53.34,803,Clouds,broken clouds,04d,stations,6.32,1.81,...,1583414286,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
6,2020-03-05 13:52,-6.27,53.34,803,Clouds,broken clouds,04d,stations,6.34,1.68,...,1583416204,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
7,2020-03-05 14:22,-6.27,53.34,803,Clouds,broken clouds,04d,stations,6.79,2.21,...,1583417827,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
8,2020-03-05 14:52,-6.27,53.34,803,Clouds,broken clouds,04d,stations,7.15,3.32,...,1583419704,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
9,2020-03-05 15:22,-6.27,53.34,803,Clouds,broken clouds,04d,stations,7.06,3.21,...,1583421692,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200


In [7]:
# Get the bike data from '2020-03-24 18:00:27'
df = df[(pd.to_datetime(df['now_time'] ,format = '%Y-%m-%d %H:%M:%S')>= pd.to_datetime('2020-03-24 18:00:27',format = '%Y-%m-%d %H:%M:%S'))]
df = df.reset_index(drop=True)
df


,number,status,available_bike_stands,available_bikes,last_update,now_time
0,42,OPEN,18,11,2020-03-24 17:59:20,2020-03-24 18:00:27
1,30,OPEN,8,12,2020-03-24 17:52:44,2020-03-24 18:00:27
2,54,OPEN,16,17,2020-03-24 17:54:32,2020-03-24 18:00:27
3,108,OPEN,23,17,2020-03-24 17:53:16,2020-03-24 18:00:27
4,56,OPEN,30,10,2020-03-24 17:57:14,2020-03-24 18:00:27
...,...,...,...,...,...,...
230885,39,OPEN,7,13,2020-04-01 01:23:37,2020-04-01 01:28:29
230886,83,OPEN,23,17,2020-04-01 01:23:42,2020-04-01 01:28:29
230887,92,OPEN,27,13,2020-04-01 01:23:57,2020-04-01 01:28:29
230888,21,OPEN,19,11,2020-04-01 01:23:48,2020-04-01 01:28:29


In [8]:
df['now_time'] = pd.to_datetime(df['now_time'])
df.info()
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230890 entries, 0 to 230889
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   number                 230890 non-null  object        
 1   status                 230890 non-null  object        
 2   available_bike_stands  230890 non-null  object        
 3   available_bikes        230890 non-null  object        
 4   last_update            230890 non-null  object        
 5   now_time               230890 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 10.6+ MB


In [9]:
# Get the weather data from '2020-03-24 18:01'
df_2 = df_2[(pd.to_datetime(df_2['current'] ,format = '%Y-%m-%d %H:%M')>= pd.to_datetime('2020-03-24 18:01',format = '%Y-%m-%d %H:%M'))]
df_2 = df_2.reset_index(drop=True)
df_2


,current,lon,lat,weather_id,main,description,icon,base,temp,feels_like,...,dt,sys_type,sys_id,sys_country,sunrise,sunset,timezone,id,name,cod
0,2020-03-24 18:01,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
1,2020-03-24 18:06,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
2,2020-03-24 18:11,-6.27,53.34,803,Clouds,broken clouds,04d,stations,10.84,4.1,...,1585073453,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
3,2020-03-24 18:16,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585073019,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
4,2020-03-24 18:21,-6.27,53.34,803,Clouds,broken clouds,04d,stations,10.84,4.1,...,1585073453,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,2020-04-01 01:05,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.17,1.83,...,1585702910,1,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200
2102,2020-04-01 01:10,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.17,1.83,...,1585703181,1,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200
2103,2020-04-01 01:15,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.17,1.83,...,1585703181,1,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200
2104,2020-04-01 01:20,-6.27,53.34,803,Clouds,broken clouds,04n,stations,5.79,2.25,...,1585704012,1,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200


In [10]:
df_2['current'] = pd.to_datetime(df_2['current'])
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2106 entries, 0 to 2105
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   current      2106 non-null   datetime64[ns]
 1   lon          2106 non-null   object        
 2   lat          2106 non-null   object        
 3   weather_id   2106 non-null   object        
 4   main         2106 non-null   object        
 5   description  2106 non-null   object        
 6   icon         2106 non-null   object        
 7   base         2106 non-null   object        
 8   temp         2106 non-null   object        
 9   feels_like   2106 non-null   object        
 10  temp_min     2106 non-null   object        
 11  temp_max     2106 non-null   object        
 12  pressure     2106 non-null   object        
 13  humidity     2106 non-null   object        
 14  visibility   2106 non-null   object        
 15  speed        2106 non-null   object        
 16  deg   

In [11]:
#according to the number of bike station, replicate the weather date for each line for further concat
def copy(df,df_2):
    df_3 = pd.DataFrame()
    count = 0
    for i in range(0,len(df)-1,110):
        if (df_2.iloc[count,0] - df.iloc[i,5]) < pd.Timedelta('0 days 00:02:00'):
            each_row = df_2.loc[count]
            row = pd.DataFrame(each_row).T
            df_3 = df_3.append([row]*110)
            
            count +=1
    
    return df_3       
df_3 = copy(df,df_2) 

In [12]:
#change the index
df_3 = df_3.reset_index(drop=True)
df_3


,current,lon,lat,weather_id,main,description,icon,base,temp,feels_like,...,dt,sys_type,sys_id,sys_country,sunrise,sunset,timezone,id,name,cod
0,2020-03-24 18:01:00,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
1,2020-03-24 18:01:00,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
2,2020-03-24 18:01:00,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
3,2020-03-24 18:01:00,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
4,2020-03-24 18:01:00,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230885,2020-04-01 00:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1585701407,1,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200
230886,2020-04-01 00:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1585701407,1,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200
230887,2020-04-01 00:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1585701407,1,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200
230888,2020-04-01 00:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1585701407,1,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200


In [ ]:
#join the two dataframe(bike and weather)
AllBikeWeather = pd.concat([df, df_3], axis=1)
AllBikeWeather

In [ ]:
for index, elem in AllBikeWeather['current'].items():
    if AllBikeWeather.loc[index,'timezone'] =="3600":
        
        a = elem  + timedelta(hours=1)
        
        AllBikeWeather.loc[index,'current'] = elem  + timedelta(hours=1)

In [27]:
AllBikeWeather['hour'] = None
AllBikeWeather['weekday'] = None
for index, elem in AllBikeWeather['current'].items():
#     d = datetime.strptime(elem, '%Y-%m-%d %H:%M:%S')

    hour = elem.hour
    weekday = elem.weekday()
    AllBikeWeather.loc[index,'hour'] = hour
    AllBikeWeather.loc[index,'weekday'] = weekday

current        2100
lon               1
lat               1
weather_id        9
main              5
description       9
icon             12
base              1
temp            573
feels_like      695
temp_min         43
temp_max         36
pressure         31
humidity         31
visibility       15
speed            22
deg               6
gust              6
clouds_all        8
dt             1531
sys_type          1
sys_id            1
sys_country       1
sunrise           9
sunset            9
timezone          2
id                1
name              1
cod               1
dtype: int64

### Clean the data



In [ ]:
AllBikeWeather.isnull().sum()


In [ ]:
AllBikeWeather.nunique()


#### Drop some null value and useless value


In [ ]:
#since the def and gust have too many null value, which is more than 70% of the whole value
#drop the cloumns which the cadinality equals 1(this kind of infomation do not tell much about the codition)
#the status = open is the same for all station
#the lon and lat is the altitide of dublin 

#the description and main show the same kind of info
#the temp and temp_min, temp_max kind of duplicate, drop two of them
drop_list = ['deg', 'gust','status','lon','lat' ,'base','description','icon','pressure','humidity','visibility','speed','temp_min','temp_max','clouds_all','dt','sys_type','sys_id','sys_country','sunrise','sunset','timezone','id','name','cod']
AllBikeWeather = AllBikeWeather.drop(columns=['deg', 'gust','status','lon','lat' ,'base','description','icon','pressure','humidity','visibility','speed','temp_min','temp_max','clouds_all','dt','sys_type','sys_id','sys_country','sunrise','sunset','timezone','id','name','cod'])


In [ ]:
AllBikeWeather


In [ ]:
AllBikeWeather.nunique()

In [ ]:
AllBikeWeather.dtypes

### Change the type of remaining features

In [ ]:
AllBikeWeather['number'] = AllBikeWeather['number'].astype('category')
AllBikeWeather['available_bike_stands'] = AllBikeWeather['available_bike_stands'].astype('int')
AllBikeWeather['available_bikes'] = AllBikeWeather['available_bikes'].astype('int')
AllBikeWeather['last_update'] = pd.to_datetime(AllBikeWeather['last_update'])

AllBikeWeather['weather_id'] = AllBikeWeather['weather_id'].astype('category')
AllBikeWeather['main'] = AllBikeWeather['main'].astype('category')
AllBikeWeather['temp'] = AllBikeWeather['temp'].astype('double')
AllBikeWeather['feels_like'] = AllBikeWeather['feels_like'].astype('double')
AllBikeWeather.dtypes

In [ ]:
df_3['hour'] = None
df_3['weekday'] = None
for index, elem in df_3['current'].items():
#     d = datetime.strptime(elem, '%Y-%m-%d %H:%M:%S')

    hour = elem.hour
    weekday = elem.weekday()
    df_3.loc[index,'hour'] = hour
    df_3.loc[index,'weekday'] = weekday

In [ ]:
df_4.dtypes

In [ ]:
df = df_3.drop(columns=['lon', 'lat','weather_id','description','icon','base','temp','temp_min','temp_max','pressure','humidity','visibility','speed','deg','gust','clouds_all','dt','sys_type','sys_id','sys_country','sunrise','sunset','timezone','id','name','cod'])

,current,lon,lat,weather_id,main,description,icon,base,temp,feels_like,...,sys_id,sys_country,sunrise,sunset,timezone,id,name,cod,hour,weekday
230880,2020-04-01 01:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200,1,2
230881,2020-04-01 01:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200,1,2
230882,2020-04-01 01:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200,1,2
230883,2020-04-01 01:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200,1,2
230884,2020-04-01 01:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200,1,2
230885,2020-04-01 01:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200,1,2
230886,2020-04-01 01:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200,1,2
230887,2020-04-01 01:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200,1,2
230888,2020-04-01 01:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200,1,2
230889,2020-04-01 01:50:00,-6.27,53.34,803,Clouds,broken clouds,04n,stations,6.36,2.05,...,1565,IE,1585720652,1585767588,3600,2964574,Dublin,200,1,2


current        datetime64[ns]
lon                    object
lat                    object
weather_id             object
main                   object
description            object
icon                   object
base                   object
temp                   object
feels_like             object
temp_min               object
temp_max               object
pressure               object
humidity               object
visibility             object
speed                  object
deg                    object
gust                   object
clouds_all             object
dt                     object
sys_type               object
sys_id                 object
sys_country            object
sunrise                object
sunset                 object
timezone               object
id                     object
name                   object
cod                    object
hour                   object
weekday                object
dtype: object

In [ ]:
d


,current,main,feels_like
0,2020-03-24 18:01:00,Clouds,3.85
1,2020-03-24 18:01:00,Clouds,3.85
2,2020-03-24 18:01:00,Clouds,3.85
3,2020-03-24 18:01:00,Clouds,3.85
4,2020-03-24 18:01:00,Clouds,3.85
5,2020-03-24 18:01:00,Clouds,3.85
6,2020-03-24 18:01:00,Clouds,3.85
7,2020-03-24 18:01:00,Clouds,3.85
8,2020-03-24 18:01:00,Clouds,3.85
9,2020-03-24 18:01:00,Clouds,3.85


In [8]:
df = df['now_time']
df

0         2020-03-24 18:00:27
1         2020-03-24 18:00:27
2         2020-03-24 18:00:27
3         2020-03-24 18:00:27
4         2020-03-24 18:00:27
                 ...         
216695    2020-03-31 14:41:19
216696    2020-03-31 14:41:19
216697    2020-03-31 14:41:19
216698    2020-03-31 14:41:19
216699    2020-03-31 14:41:19
Name: now_time, Length: 216700, dtype: object

In [13]:
mergedStuff = pd.merge(df, df_2, left_on = 'current', how = 'outer')
mergedStuff

TypeError: object of type 'NoneType' has no len()

In [ ]:
d = pd.DataFrame({'A':{'1':'01','2':'02','3':'03','4':'01','5':'02','6':'03','7':'01','8':'02','9':'03'},'ID':{'1':'10','2':'10','3':'10','4':'15','5':'15','6':'15','7':'20','8':'20','9':'20'},'C':{'1':'10A','2':'10A','3':'10A','4':'15A','5':'15A','6':'15A','7':'20A','8':'20A','9':'20A'}, })
d['D'] = d['ID']
d





In [23]:
f = pd.DataFrame({'ID':{'1':'10A','2':'15A','3':'20A'}})
o = pd.DataFrame()
for i in range(len(f)):
    a=f.iloc[i]
    k=pd.DataFrame(a).T
    o=o.append([k]*5)
o 
   

,ID
1,10A
1,10A
1,10A
1,10A
1,10A
2,15A
2,15A
2,15A
2,15A
2,15A


In [24]:
f.iloc[0]['ID']

'10A'

In [6]:
d.merge(f, how='left', on="ID")

,A,ID,D
0,01,10,10
1,02,10,10
2,03,10,10
3,01,15,15
4,02,15,15
5,03,15,15
6,01,20,20
7,02,20,20
8,03,20,20


In [4]:
def join(d1,d2):
    count =0
    for i in range(0, len(d1)-1):
        if d1.iloc[i]['D'] == d1.iloc[i+1]['D']:
            d1.loc[i,'D'] = d2.iloc[count]['ID']
            print(d1.iloc[i]['D'],d2.iloc[count]['ID'])
        else:
            d1.loc[i,'D'] = d2.iloc[count]['ID']
            count += 1
#         if d1.iloc[i]['ID'] == d1.iloc[i+1]['ID']:
#             d1.iloc[i]['D'] = d2.iloc[count]['ID']
#             print(d1.loc[i]['D'])
#         else:
#             count+=1
            
join(d,f)


10 10A
10 10A
15 15A
15 15A
20 20A
20 20A


In [5]:
d


,A,ID,D
1,01,10,10
2,02,10,10
3,03,10,10
4,01,15,15
5,02,15,15
6,03,15,15
7,01,20,20
8,02,20,20
9,03,20,20
0,NaN,NaN,10A


In [68]:
a = pd.merge(d, f,left_index=True, right_index=True, how='inner')
a

,A,B_x,B_y
1,01,10,10A
2,02,10,15A
3,03,10,20A


In [66]:
a = [(d, f) for k, l in zip(ntest, ltest) for v in l]

NameError: name 'ntest' is not defined